載入資料

In [1]:
import pandas as pd
import numpy as np
import re
import string
from collections import defaultdict
import logging
from gensim.models import word2vec

from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
path = "./ml-2017fall-hw4/"

train = pd.read_csv(path + "training_label.txt", sep="\+\+\+\$\+\+\+", header=None, names=["target", "text"])

/home/wugu504/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
train.head()

,target,text
0,1,are wtf ... awww thanks !
1,1,leavingg to wait for kaysie to arrive myspaci...
2,0,i wish i could go and see duffy when she come...
3,1,i know eep ! i can ' t wait for one more day ...
4,0,so scared and feeling sick . fuck ! hope some...


In [4]:
nolabel = pd.read_csv(path + "training_nolabel.txt", header=None, index_col=False, names=["text"])
nolabel.head()

,text
0,mkhang mlbo . dami niang followers ee . di q r...
1,don ' t you hate it when you hang on to a seem...
2,ok so never went to the movies because friend ...
3,can ' t wait to see diversity ' s performance !
4,i love britney spears haha joey this is what u...


In [5]:
path = "./ml-2017fall-hw4/"
test = pd.read_csv(path + "testing_data.txt", sep="^([^,]+),", header=0, index_col=False).iloc[:,1:]
test.head()

/home/wugu504/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,id,text
0,0,"my dog ate our dinner . no , seriously ... he ..."
1,1,omg last day sooon n of primary noooooo x im g...
2,2,stupid boys .. they ' re so .. stupid !
3,3,hi ! do u know if the nurburgring is open for ...
4,4,"having lunch in the office , and thinking of h..."


In [6]:
wordlist = list(train["text"]) + list(nolabel["text"]) + list(test["text"])
wordlist[:5]

[' are wtf ... awww thanks !',
 ' leavingg to wait for kaysie to arrive myspacin itt for now ilmmthek .!',
 ' i wish i could go and see duffy when she comes to mamaia romania .',
 " i know eep ! i can ' t wait for one more day ....",
 ' so scared and feeling sick . fuck ! hope someone at hr help ... wish it would be wendita or karen .']

全部轉為小寫

In [7]:
wordlist = [str(x).lower() for x in wordlist]
wordlist[:5]

[' are wtf ... awww thanks !',
 ' leavingg to wait for kaysie to arrive myspacin itt for now ilmmthek .!',
 ' i wish i could go and see duffy when she comes to mamaia romania .',
 " i know eep ! i can ' t wait for one more day ....",
 ' so scared and feeling sick . fuck ! hope someone at hr help ... wish it would be wendita or karen .']

移除數字

In [8]:
wordlist = [re.sub(r"\d+", "", x) for x in wordlist]
wordlist[:5]

[' are wtf ... awww thanks !',
 ' leavingg to wait for kaysie to arrive myspacin itt for now ilmmthek .!',
 ' i wish i could go and see duffy when she comes to mamaia romania .',
 " i know eep ! i can ' t wait for one more day ....",
 ' so scared and feeling sick . fuck ! hope someone at hr help ... wish it would be wendita or karen .']

移除標點符號

In [9]:
wordlist = [x.translate(str.maketrans("", "", string.punctuation)) for x in wordlist]
wordlist[:5]

[' are wtf  awww thanks ',
 ' leavingg to wait for kaysie to arrive myspacin itt for now ilmmthek ',
 ' i wish i could go and see duffy when she comes to mamaia romania ',
 ' i know eep  i can  t wait for one more day ',
 ' so scared and feeling sick  fuck  hope someone at hr help  wish it would be wendita or karen ']

移除前後的空格

In [10]:
wordlist = [x.strip() for x in wordlist]
wordlist[:5]

['are wtf  awww thanks',
 'leavingg to wait for kaysie to arrive myspacin itt for now ilmmthek',
 'i wish i could go and see duffy when she comes to mamaia romania',
 'i know eep  i can  t wait for one more day',
 'so scared and feeling sick  fuck  hope someone at hr help  wish it would be wendita or karen']

將句子中的字詞拆解

In [11]:
texts = [[word for word in text.split()] for text in wordlist]
texts[:5]

[['are', 'wtf', 'awww', 'thanks'],
 ['leavingg',
  'to',
  'wait',
  'for',
  'kaysie',
  'to',
  'arrive',
  'myspacin',
  'itt',
  'for',
  'now',
  'ilmmthek'],
 ['i',
  'wish',
  'i',
  'could',
  'go',
  'and',
  'see',
  'duffy',
  'when',
  'she',
  'comes',
  'to',
  'mamaia',
  'romania'],
 ['i', 'know', 'eep', 'i', 'can', 't', 'wait', 'for', 'one', 'more', 'day'],
 ['so',
  'scared',
  'and',
  'feeling',
  'sick',
  'fuck',
  'hope',
  'someone',
  'at',
  'hr',
  'help',
  'wish',
  'it',
  'would',
  'be',
  'wendita',
  'or',
  'karen']]

去掉只出現一次的單詞

In [12]:
frequency = defaultdict(int)

for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]
texts[:5]

[['are', 'wtf', 'awww', 'thanks'],
 ['leavingg',
  'to',
  'wait',
  'for',
  'to',
  'arrive',
  'myspacin',
  'itt',
  'for',
  'now'],
 ['i',
  'wish',
  'i',
  'could',
  'go',
  'and',
  'see',
  'duffy',
  'when',
  'she',
  'comes',
  'to',
  'mamaia',
  'romania'],
 ['i', 'know', 'eep', 'i', 'can', 't', 'wait', 'for', 'one', 'more', 'day'],
 ['so',
  'scared',
  'and',
  'feeling',
  'sick',
  'fuck',
  'hope',
  'someone',
  'at',
  'hr',
  'help',
  'wish',
  'it',
  'would',
  'be',
  'wendita',
  'or',
  'karen']]

儲存成字典

In [13]:
# sg=1 means Skip-gram 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
w2vmodel = word2vec.Word2Vec(texts, size=200, sg=1, min_count=1, window=3, workers=6, iter=20)

2019-04-03 22:01:28,151 : INFO : collecting all words and their counts
2019-04-03 22:01:28,152 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-04-03 22:01:28,169 : INFO : PROGRESS: at sentence #10000, processed 130409 words, keeping 12779 word types
2019-04-03 22:01:28,186 : INFO : PROGRESS: at sentence #20000, processed 260264 words, keeping 19206 word types
2019-04-03 22:01:28,205 : INFO : PROGRESS: at sentence #30000, processed 390139 words, keeping 24259 word types
2019-04-03 22:01:28,224 : INFO : PROGRESS: at sentence #40000, processed 520247 words, keeping 28556 word types
2019-04-03 22:01:28,243 : INFO : PROGRESS: at sentence #50000, processed 649085 words, keeping 32366 word types
2019-04-03 22:01:28,259 : INFO : PROGRESS: at sentence #60000, processed 779859 words, keeping 35858 word types
2019-04-03 22:01:28,277 : INFO : PROGRESS: at sentence #70000, processed 910947 words, keeping 39113 word types
2019-04-03 22:01:28,296 : INFO : PROGRESS: at 

2019-04-03 22:01:29,304 : INFO : PROGRESS: at sentence #720000, processed 8127600 words, keeping 100966 word types
2019-04-03 22:01:29,320 : INFO : PROGRESS: at sentence #730000, processed 8233858 words, keeping 101342 word types
2019-04-03 22:01:29,334 : INFO : PROGRESS: at sentence #740000, processed 8340314 words, keeping 101729 word types
2019-04-03 22:01:29,350 : INFO : PROGRESS: at sentence #750000, processed 8446382 words, keeping 102093 word types
2019-04-03 22:01:29,364 : INFO : PROGRESS: at sentence #760000, processed 8551312 words, keeping 102484 word types
2019-04-03 22:01:29,381 : INFO : PROGRESS: at sentence #770000, processed 8658386 words, keeping 102847 word types
2019-04-03 22:01:29,397 : INFO : PROGRESS: at sentence #780000, processed 8764306 words, keeping 103193 word types
2019-04-03 22:01:29,411 : INFO : PROGRESS: at sentence #790000, processed 8871070 words, keeping 103523 word types
2019-04-03 22:01:29,426 : INFO : PROGRESS: at sentence #800000, processed 897760

2019-04-03 22:01:30,414 : INFO : PROGRESS: at sentence #1430000, processed 15874847 words, keeping 117182 word types
2019-04-03 22:01:30,432 : INFO : PROGRESS: at sentence #1440000, processed 16003952 words, keeping 117193 word types
2019-04-03 22:01:30,451 : INFO : PROGRESS: at sentence #1450000, processed 16133626 words, keeping 117205 word types
2019-04-03 22:01:30,468 : INFO : PROGRESS: at sentence #1460000, processed 16264033 words, keeping 117217 word types
2019-04-03 22:01:30,485 : INFO : PROGRESS: at sentence #1470000, processed 16392236 words, keeping 117223 word types
2019-04-03 22:01:30,507 : INFO : PROGRESS: at sentence #1480000, processed 16521008 words, keeping 117235 word types
2019-04-03 22:01:30,524 : INFO : PROGRESS: at sentence #1490000, processed 16650611 words, keeping 117247 word types
2019-04-03 22:01:30,543 : INFO : PROGRESS: at sentence #1500000, processed 16781127 words, keeping 117257 word types
2019-04-03 22:01:30,560 : INFO : PROGRESS: at sentence #1510000,

2019-04-03 22:02:10,315 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-04-03 22:02:10,321 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-04-03 22:02:10,323 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-04-03 22:02:10,323 : INFO : EPOCH - 3 : training on 17408254 raw words (13499875 effective words) took 13.0s, 1035195 effective words/s
2019-04-03 22:02:11,332 : INFO : EPOCH 4 - PROGRESS: at 5.71% examples, 889515 words/s, in_qsize 11, out_qsize 0
2019-04-03 22:02:12,334 : INFO : EPOCH 4 - PROGRESS: at 11.74% examples, 912116 words/s, in_qsize 11, out_qsize 0
2019-04-03 22:02:13,336 : INFO : EPOCH 4 - PROGRESS: at 18.46% examples, 904762 words/s, in_qsize 11, out_qsize 0
2019-04-03 22:02:14,336 : INFO : EPOCH 4 - PROGRESS: at 25.81% examples, 912974 words/s, in_qsize 11, out_qsize 0
2019-04-03 22:02:15,339 : INFO : EPOCH 4 - PROGRESS: at 33.07% examples, 915782 words/s, in_qsize 11, out_qsize 0
2019-04-03 22:02:

2019-04-03 22:03:01,837 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-04-03 22:03:01,838 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-04-03 22:03:01,844 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-04-03 22:03:01,849 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-04-03 22:03:01,849 : INFO : EPOCH - 7 : training on 17408254 raw words (13496376 effective words) took 12.1s, 1112109 effective words/s
2019-04-03 22:03:02,861 : INFO : EPOCH 8 - PROGRESS: at 6.91% examples, 1074076 words/s, in_qsize 11, out_qsize 0
2019-04-03 22:03:03,868 : INFO : EPOCH 8 - PROGRESS: at 14.33% examples, 1091087 words/s, in_qsize 11, out_qsize 0
2019-04-03 22:03:04,870 : INFO : EPOCH 8 - PROGRESS: at 23.25% examples, 1106515 words/s, in_qsize 12, out_qsize 0
2019-04-03 22:03:05,875 : INFO : EPOCH 8 - PROGRESS: at 32.10% examples, 1111227 words/s, in_qsize 11, out_qsize 0
2019-04-03 22:03:06,880 : INFO : EPO

2019-04-03 22:03:51,692 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-04-03 22:03:51,704 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-04-03 22:03:51,711 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-04-03 22:03:51,718 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-04-03 22:03:51,722 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-04-03 22:03:51,726 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-04-03 22:03:51,726 : INFO : EPOCH - 11 : training on 17408254 raw words (13497398 effective words) took 13.6s, 993706 effective words/s
2019-04-03 22:03:52,735 : INFO : EPOCH 12 - PROGRESS: at 6.96% examples, 1082721 words/s, in_qsize 12, out_qsize 0
2019-04-03 22:03:53,741 : INFO : EPOCH 12 - PROGRESS: at 14.33% examples, 1092503 words/s, in_qsize 11, out_qsize 0
2019-04-03 22:03:54,747 : INFO : EPOCH 12 - PROGRESS: at 23.13% examples, 1100568 

2019-04-03 22:04:42,476 : INFO : EPOCH 15 - PROGRESS: at 95.36% examples, 1153661 words/s, in_qsize 11, out_qsize 0
2019-04-03 22:04:43,073 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-04-03 22:04:43,078 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-04-03 22:04:43,082 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-04-03 22:04:43,086 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-04-03 22:04:43,091 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-04-03 22:04:43,097 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-04-03 22:04:43,098 : INFO : EPOCH - 15 : training on 17408254 raw words (13498154 effective words) took 11.7s, 1153975 effective words/s
2019-04-03 22:04:44,113 : INFO : EPOCH 16 - PROGRESS: at 7.21% examples, 1113349 words/s, in_qsize 11, out_qsize 0
2019-04-03 22:04:45,115 : INFO : EPOCH 16 - PROGRESS: at 14.93% examples, 1128923

2019-04-03 22:05:31,232 : INFO : EPOCH 19 - PROGRESS: at 99.64% examples, 1115508 words/s, in_qsize 8, out_qsize 0
2019-04-03 22:05:31,259 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-04-03 22:05:31,261 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-04-03 22:05:31,272 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-04-03 22:05:31,273 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-04-03 22:05:31,277 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-04-03 22:05:31,292 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-04-03 22:05:31,293 : INFO : EPOCH - 19 : training on 17408254 raw words (13500044 effective words) took 12.1s, 1114477 effective words/s
2019-04-03 22:05:32,303 : INFO : EPOCH 20 - PROGRESS: at 6.91% examples, 1074322 words/s, in_qsize 11, out_qsize 0
2019-04-03 22:05:33,309 : INFO : EPOCH 20 - PROGRESS: at 14.27% examples, 1087708 

In [14]:
w2vmodel.save("w2vmodel.model")

2019-04-03 22:07:11,370 : INFO : saving Word2Vec object under w2vmodel.model, separately None
2019-04-03 22:07:11,371 : INFO : storing np array 'vectors' to w2vmodel.model.wv.vectors.npy
2019-04-03 22:07:11,548 : INFO : not storing attribute vectors_norm
2019-04-03 22:07:11,552 : INFO : storing np array 'syn1neg' to w2vmodel.model.trainables.syn1neg.npy
2019-04-03 22:07:11,713 : INFO : not storing attribute cum_table
2019-04-03 22:07:11,902 : INFO : saved w2vmodel.model


In [15]:
w2vmodel = word2vec.Word2Vec.load("w2vmodel.model")

2019-04-03 22:07:13,715 : INFO : loading Word2Vec object from w2vmodel.model
2019-04-03 22:07:13,821 : INFO : loading wv recursively from w2vmodel.model.wv.* with mmap=None
2019-04-03 22:07:13,822 : INFO : loading vectors from w2vmodel.model.wv.vectors.npy with mmap=None
2019-04-03 22:07:13,854 : INFO : setting ignored attribute vectors_norm to None
2019-04-03 22:07:13,854 : INFO : loading vocabulary recursively from w2vmodel.model.vocabulary.* with mmap=None
2019-04-03 22:07:13,855 : INFO : loading trainables recursively from w2vmodel.model.trainables.* with mmap=None
2019-04-03 22:07:13,855 : INFO : loading syn1neg from w2vmodel.model.trainables.syn1neg.npy with mmap=None
2019-04-03 22:07:13,886 : INFO : setting ignored attribute cum_table to None
2019-04-03 22:07:13,887 : INFO : loaded w2vmodel.model


隨便測試一個字看看！

In [16]:
w2vmodel.wv["computer"]

array([ 0.2597136 ,  0.0680491 ,  0.14870939, -0.06785405,  0.01889782,
       -0.18520711, -0.1885602 , -0.0995933 ,  0.35392967,  0.1888192 ,
       -0.25261167,  0.11654913, -0.44418007,  0.2067864 , -0.15940247,
       -0.14691448, -0.05473197,  0.39358056,  0.01704257,  0.12057644,
       -0.3107193 ,  0.42072743, -0.01560979, -0.41902515,  0.73674464,
        0.37681928,  0.27171925,  0.14049773,  0.23618628, -0.2943771 ,
       -0.08489822,  0.4050786 , -0.2586974 , -0.11106346,  0.29834905,
        0.7025178 ,  0.00437329, -0.4504733 , -0.45381746,  0.03983081,
       -0.19527617, -0.26875544,  0.2207269 , -0.07695176, -0.02419234,
       -0.00083169, -0.00693199, -0.07137527,  0.28264153, -0.09566917,
        0.3498552 ,  0.04825199,  0.16344097, -0.1640132 , -0.3556715 ,
        0.24367857,  0.7240531 , -0.47524574,  0.28263304, -0.30293864,
       -0.4570374 , -0.13742852, -0.1785007 ,  0.10404795,  0.03954203,
        0.11146531, -0.02262113, -0.15684307, -0.5676298 ,  0.04

In [17]:
len(w2vmodel.wv.vocab.items())

117286

這是weight

In [18]:
w2vmodel.wv.vectors

array([[-0.03790323, -0.25120223, -0.06700747, ..., -0.12902237,
         0.14676812, -0.40879408],
       [-0.21863802, -0.15415996,  0.02521611, ...,  0.12911823,
         0.20452441, -0.2958011 ],
       [ 0.12032862, -0.17554872,  0.04350717, ..., -0.01508772,
         0.01840604, -0.17322086],
       ...,
       [-0.00238501,  0.05989103,  0.11711463, ..., -0.07439116,
         0.17107767, -0.1232437 ],
       [-0.12950729,  0.03237965,  0.14300607, ..., -0.04196416,
         0.2047708 , -0.14363886],
       [-0.08707895, -0.11553323,  0.10134102, ...,  0.00723493,
         0.2708728 , -0.13376625]], dtype=float32)

In [19]:
vocab_size, embedding_size = w2vmodel.wv.vectors.shape
w2vmodel.wv.vectors.shape

(117286, 200)

In [20]:
def wordtoidx(word):
    if word not in w2vmodel.wv.vocab:
        return 0
    else: 
        return w2vmodel.wv.vocab[word].index
def idxtoword(idx):
    return w2vmodel.wv.index2word[idx]

In [21]:
word2idx = {"_PAD": 0} 

# 多一位PADDING = 0
embeddings_matrix = np.zeros((vocab_size + 1, embedding_size))
for i in range(vocab_size):
    word = idxtoword(i)
    word2idx[word] = i + 1
    embeddings_matrix[i + 1] = w2vmodel.wv.vectors[i]

In [22]:
embeddings_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.03790323, -0.25120223, -0.06700747, ..., -0.12902237,
         0.14676812, -0.40879408],
       [-0.21863802, -0.15415996,  0.02521611, ...,  0.12911823,
         0.20452441, -0.2958011 ],
       ...,
       [-0.00238501,  0.05989103,  0.11711463, ..., -0.07439116,
         0.17107767, -0.1232437 ],
       [-0.12950729,  0.03237965,  0.14300607, ..., -0.04196416,
         0.2047708 , -0.14363886],
       [-0.08707895, -0.11553323,  0.10134102, ...,  0.00723493,
         0.2708728 , -0.13376625]])

跟原本的對照一下！

In [23]:
w2vmodel.wv.vectors

array([[-0.03790323, -0.25120223, -0.06700747, ..., -0.12902237,
         0.14676812, -0.40879408],
       [-0.21863802, -0.15415996,  0.02521611, ...,  0.12911823,
         0.20452441, -0.2958011 ],
       [ 0.12032862, -0.17554872,  0.04350717, ..., -0.01508772,
         0.01840604, -0.17322086],
       ...,
       [-0.00238501,  0.05989103,  0.11711463, ..., -0.07439116,
         0.17107767, -0.1232437 ],
       [-0.12950729,  0.03237965,  0.14300607, ..., -0.04196416,
         0.2047708 , -0.14363886],
       [-0.08707895, -0.11553323,  0.10134102, ...,  0.00723493,
         0.2708728 , -0.13376625]], dtype=float32)

In [24]:
embeddings_matrix.shape

(117287, 200)

建立ＬＳＴＭ模型型型！！！

In [25]:
lstmmodel = Sequential()
lstmmodel.add(Embedding(input_dim=vocab_size + 1, output_dim=embedding_size, \
                   weights=[embeddings_matrix], trainable=False))
lstmmodel.add(LSTM(units=embedding_size, return_sequences=True, dropout=0.4))
lstmmodel.add(LSTM(units=embedding_size, dropout=0.4))
lstmmodel.add(Dense(units=1)) # output 只有1個neuron，正評或負評
lstmmodel.add(Activation("sigmoid"))

lstmmodel.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

lstmmodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 200)         23457400  
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 200)         320800    
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 24,099,201
Trainable params: 641,801
Non-trainable params: 23,457,400
_________________________________________________________________


準備訓練資料集！前200000行為訓練資料集，將每個字轉到對應的id

In [26]:
trainlist = texts[:200000]
max_length = max([len(x) for x in trainlist])
max_length

37

In [27]:
target = np.array(train["target"])
target

array([1, 1, 0, ..., 0, 1, 0])

In [28]:
train_x = np.zeros([len(trainlist), max_length], dtype=np.int32)
train_y = np.zeros([len(trainlist)], dtype=np.int32)
for i, sentence in enumerate(trainlist):
    for t, word in enumerate(sentence):
        train_x[i, t] = wordtoidx(word)
        train_y[i] = target[i]

看一下第一行！

In [29]:
wordlist[0]

'are wtf  awww thanks'

In [30]:
train_x[0]

array([ 36, 752, 411,  76,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

真正要開始訓練了！！！！

In [31]:
early_stopping = EarlyStopping(monitor="val_acc", patience=5, verbose=1, mode="max")

In [32]:
lstmmodel.fit(train_x, train_y, batch_size=500, epochs=40, verbose=1, validation_split=0.1, shuffle=True, \
             callbacks=[early_stopping])

Train on 180000 samples, validate on 20000 samples
Epoch 1/40
180000/180000 [==============================] - 22s 123us/step - loss: 0.6480 - acc: 0.6184 - val_loss: 0.5777 - val_acc: 0.6963
Epoch 2/40
180000/180000 [==============================] - 21s 118us/step - loss: 0.5586 - acc: 0.7099 - val_loss: 0.5272 - val_acc: 0.7446
Epoch 3/40
180000/180000 [==============================] - 21s 118us/step - loss: 0.5242 - acc: 0.7378 - val_loss: 0.5002 - val_acc: 0.7573
Epoch 4/40
180000/180000 [==============================] - 21s 118us/step - loss: 0.5075 - acc: 0.7506 - val_loss: 0.4820 - val_acc: 0.7692
Epoch 5/40
180000/180000 [==============================] - 21s 118us/step - loss: 0.4945 - acc: 0.7597 - val_loss: 0.5074 - val_acc: 0.7438
Epoch 6/40
180000/180000 [==============================] - 21s 118us/step - loss: 0.4827 - acc: 0.7679 - val_loss: 0.4677 - val_acc: 0.7779
Epoch 7/40
180000/180000 [==============================] - 21s 118us/step - loss: 0.4758 - acc: 0.7715

In [33]:
lstmmodel.save("lstmmodel.h5")

載入模型

tensorflow==1.12.0<br>
keras==2.2.4

In [34]:
lstmmodel = load_model("lstmmodel.h5")

載入測試資料

In [36]:
testlist = texts[-200000:]
testmax_length = max([len(x) for x in testlist])
testmax_length

39

In [37]:
test_x = np.zeros([len(testlist), testmax_length], dtype=np.int32)

for i, sentence in enumerate(testlist):
    for t, word in enumerate(sentence):
        test_x[i, t] = wordtoidx(word)

In [38]:
test_x = test_x[:,:max_length]

In [39]:
lstmmodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 200)         23457400  
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 200)         320800    
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 24,099,201
Trainable params: 641,801
Non-trainable params: 23,457,400
_________________________________________________________________


In [41]:
predclass = lstmmodel.predict(test_x, batch_size=500, verbose=1)

200000/200000 [==============================] - 9s 44us/step


In [42]:
predclass

array([[0.02000418],
       [0.05712306],
       [0.05215343],
       ...,
       [0.62821865],
       [0.06688549],
       [0.06750049]], dtype=float32)

In [43]:
sum(predclass >= 0.5)

array([94935])

In [44]:
submit = pd.read_csv(path + "sampleSubmission.csv")
submit.head()

,id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [45]:
submit["label"] = predclass

In [46]:
submit.head()

,id,label
0,0,0.020004
1,1,0.057123
2,2,0.052153
3,3,0.492487
4,4,0.220749


In [47]:
submit["label"] = submit["label"].apply(lambda x: 1 if x >= 0.5 else 0)
submit.head()

,id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [48]:
sum(submit["label"] == 1)

94935

In [49]:
submit.to_csv("newgpu_submit4.csv", index=False)